# Exercises - Pricing a Callable Bond


#### Notation Commands

$$\newcommand{\Black}{\mathcal{B}}
\newcommand{\Blackcall}{\Black_{\mathrm{call}}}
\newcommand{\Blackput}{\Black_{\mathrm{put}}}
\newcommand{\EcondS}{\hat{S}_{\mathrm{conditional}}}
\newcommand{\Efwd}{\mathbb{E}^{T}}
\newcommand{\Ern}{\mathbb{E}^{\mathbb{Q}}}
\newcommand{\Tfwd}{T_{\mathrm{fwd}}}
\newcommand{\Tunder}{T_{\mathrm{bond}}}
\newcommand{\accint}{A}
\newcommand{\carry}{\widetilde{\cpn}}
\newcommand{\cashflow}{C}
\newcommand{\convert}{\phi}
\newcommand{\cpn}{c}
\newcommand{\ctd}{\mathrm{CTD}}
\newcommand{\disc}{Z}
\newcommand{\done}{d_{1}}
\newcommand{\dt}{\Delta t}
\newcommand{\dtwo}{d_{2}}
\newcommand{\flatvol}{\sigma_{\mathrm{flat}}}
\newcommand{\flatvolT}{\sigma_{\mathrm{flat},T}}
\newcommand{\float}{\mathrm{flt}}
\newcommand{\freq}{m}
\newcommand{\futprice}{\mathcal{F}(t,T)}
\newcommand{\futpriceDT}{\mathcal{F}(t+h,T)}
\newcommand{\futpriceT}{\mathcal{F}(T,T)}
\newcommand{\futrate}{\mathscr{f}}
\newcommand{\fwdprice}{F(t,T)}
\newcommand{\fwdpriceDT}{F(t+h,T)}
\newcommand{\fwdpriceT}{F(T,T)}
\newcommand{\fwdrate}{f}
\newcommand{\fwdvol}{\sigma_{\mathrm{fwd}}}
\newcommand{\fwdvolTi}{\sigma_{\mathrm{fwd},T_i}}
\newcommand{\grossbasis}{B}
\newcommand{\hedge}{\Delta}
\newcommand{\ivol}{\sigma_{\mathrm{imp}}}
\newcommand{\logprice}{p}
\newcommand{\logyield}{y}
\newcommand{\mat}{(n)}
\newcommand{\nargcond}{d_{1}}
\newcommand{\nargexer}{d_{2}}
\newcommand{\netbasis}{\tilde{\grossbasis}}
\newcommand{\normcdf}{\mathcal{N}}
\newcommand{\notional}{K}
\newcommand{\pfwd}{P_{\mathrm{fwd}}}
\newcommand{\pnl}{\Pi}
\newcommand{\price}{P}
\newcommand{\probexer}{\hat{\mathcal{P}}_{\mathrm{exercise}}}
\newcommand{\pvstrike}{K^*}
\newcommand{\refrate}{r^{\mathrm{ref}}}
\newcommand{\rrepo}{r^{\mathrm{repo}}}
\newcommand{\spotrate}{r}
\newcommand{\spread}{s}
\newcommand{\strike}{K}
\newcommand{\swap}{\mathrm{sw}}
\newcommand{\swaprate}{\cpn_{\swap}}
\newcommand{\tbond}{\mathrm{fix}}
\newcommand{\ttm}{\tau}
\newcommand{\value}{V}
\newcommand{\vega}{\nu}
\newcommand{\years}{\tau}
\newcommand{\yearsACT}{\tau_{\mathrm{act/360}}}
\newcommand{\yield}{Y}$$


In [79]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.optimize import brentq

# 1. Black's Formula for Bond Options


## 1.1.

Consider a bond with:
* `T=3`
* face value of `N=100`
* coupons at `annual` frequency
* annualized coupon rate of `cpn=6%`.

Use the bond-pricing formula along with the discount rates in the data file to price this bond.


In [57]:
rates = pd.read_excel('discount_curve_2025-02-13.xlsx')

T = 3
N = 100
freq = 1
cpn = 0.06

rates_df = rates.loc[:, ['ttm', 'discount']]
ts = np.linspace(freq, T, T*freq)
price = 0
for i in ts:
    if i == ts[-1]:
        price += 100*rates_df.loc[rates_df['ttm'] == i, 'discount'].values[0]*(1+cpn)
    else:
        price += 100*rates_df.loc[rates_df['ttm'] == i, 'discount'].values[0]*cpn
print("Bond price:", price)

Bond price: 104.98670645677136


## 1.2.

Suppose the bond is callable by the issuer.

* `European` style
* expiration of `Topt=1.5`
* (clean) `strike=100`
* vol of `2.68%`
* forward price of `103.31.`

What is the value of the issuer's call option?


In [86]:
Topt = 1.5
strike = 100
vol = 0.0268
forward_p = 103.31

d1 = (np.log(forward_p/strike) + 0.5*vol**2*Topt)/(vol*Topt**.5)
d2 = d1 - vol*Topt**.5
discount =  rates.loc[rates['ttm'] == Topt].loc[:, 'discount'].values[0]

call_price = discount*(forward_p*norm.cdf(d1) - strike*norm.cdf(d2))
print('Call price:', call_price)

Call price: 3.373836732035071


## 1.3.

What is the price of the callable bond? 

The **callable** bond is the bond issued with an embedded call option (long the issuer.) Thus, it is the value of the vanilla bond minus the value of the call option.


In [78]:
print("Callable bond price = ", price - call_price)

Callable bond price =  101.61286972473629


## 1.4.

Which assumptions of Black's formula do we prefer to Black-Scholes for this problem?


**Black's model is preferable as it preserves no-arbitrage, is consistent with interest rate derivative modeling, and ensures positivity of the forwards while Black-Scholes has a drift assumption that is not correct for this case. Additionally, Black's prices under the forward measure and discounting uses the observed discount factors. This makes the term-structure consistent. It's also convenient that for Black's we can use the clean forward price and clean strike price directly while for Black-Scholes we would need the spot dirty price.**

## 1.5

Redo 1.2. Suppose the market prices the call option at `3.50`.

Solve for the implied volatility.


In [90]:
def black_call(F, K, sigma, r, tau):    
    d1 = (np.log(F / K) + 0.5 * sigma**2 * tau) / (sigma * np.sqrt(tau))
    d2 = d1 - sigma * np.sqrt(tau)
    discount = r
    return discount * (F * norm.cdf(d1) - K * norm.cdf(d2))

def obj(Ivol):
    return black_call(forward_p, strike, Ivol, discount, Topt) - 3.5

implied_vol = brentq(obj, 1e-6, 1)
print("Implied vol for call price = 3.5:", implied_vol)

Implied vol for call price = 3.5: 0.03094057783932469
